# Conformation et rognage de l'image SWI

In [29]:
import nibabel as nb
import nibabel.processing  as nip
import numpy as np
from scipy import ndimage
from copy import deepcopy

img = nb.loadsave.load("../share/image/input.nii")

final_dimensions = (160, 214, 176)
voxel_size = (1.0 , 1.0, 1.0)
resampled_dimensions = (256 , 256, 256) # half the original image because twice the voxel size...

orientation ='RAS'

On écrit l'image après conformation à 1 mm pour la dimension finale 160x214x176:

In [30]:
resampled =nip.conform(
    img,  out_shape=final_dimensions, voxel_size=voxel_size, order=3 ,
    cval=0.0, orientation=orientation, out_class=None)
nb.loadsave.save(resampled , "../share/image/conformed_final_dimensions.nii.gz")

...mais en pratique on va travailler en deux étapes car on veut ajuster le champ de vue de l'image finale pour être sur le cerveau et pas le cou, où pour ne pas que ça coupe si on a une grosse tête. 
Donc on écrit d'abord l'image après conformation à 1 mm, mais pour le même champ de vue que l'image originale (2 fois moins de voxels deux fois plus gros, 256x256x256, one ne perd rien):

In [31]:
orientation2 ='RAS'
resampled256 =nip.conform(
    img,  out_shape=resampled_dimensions, voxel_size=voxel_size, order=3 ,
    cval=0.0, orientation=orientation, out_class=None)
nb.loadsave.save(resampled256 , "../share/image/conformed256.nii.gz")

On va maintenant extraire un bloc aux bonnes dimensions 160x214x176 de l'image à 256x256x256 pour 1mm3 de taille de voxel. On veut centrer ce bloc sur le cerveau (ici on va directement prendre le centre de masse de l'image, pour faire simple).

Et on va décaler le référentiel de coordonnées par rapport à l'image originale pour conserver l'alignement des images.

In [32]:
# calcul du centre de gravité de l'image, on arrondit et on convertit en entiers
CdG_ijk = np.ceil(np.array(
    ndimage.measurements.center_of_mass(resampled256.get_fdata()))).astype(int)

# on va centrer le bloc sur le centre de gravité, donc on coupe la taille en 2
halfs = np.array(final_dimensions) / 2
halfs = halfs.astype(int) # on a besoin d'entiers car cela sertpour des indices du "tableau de voxels"
bbox1 = CdG_ijk - halfs  # les ijk du voxel le plus bas de la boite
bbox2 = halfs + CdG_ijk # les ijk voxel le plus haut de la bounding box

# on extrait la boite i1 -> i2, j1 -> j2, k1 -> k2 (on "slice", quoi) 
array_out = resampled256.get_fdata()[bbox1[0]:bbox2[0], bbox1[1]:bbox2[1], bbox1[2]:bbox2[2]]

# on corrige les coordonnées, donc d'abord il faut
# convertir ijk vers xyz pour la demi taille du bloc et le centre de gravité
CdG_xyz = resampled256.affine @ np.append(CdG_ijk, 1)
halfs_xyz = resampled256.affine @ np.append(halfs, 1)
affine_out = deepcopy(resampled256.affine) # on recopie la matrice affine de l'image, on va la modifier

# on décale le centre du référentiel de l'image car on part de moins loin
# (l'image est plus petite)
# et le centre n'est plus tout à fait le même 
affine_out[:,3] = affine_out[:,3] + CdG_xyz - halfs_xyz 

# on écrit l'image de résultats
cropped = nip.Nifti1Image(array_out, affine_out)
nb.loadsave.save(cropped, "../share/image/conformed256_cropped.nii.gz")